In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pdb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
imdb_data = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
print("shape -> ", imdb_data.shape)
imdb_data.head()

shape ->  (50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [4]:
imdb_data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [5]:
train_reviews = imdb_data.review[:40000]
test_reviews = imdb_data.review[:40000]

train_sentiments = imdb_data.sentiment[40000:]
test_sentiments = imdb_data.sentiment[40000:]

print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (10000,)
(40000,) (10000,)


In [6]:
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

tokenizer =  ToktokTokenizer()
stopwords_list = nltk.corpus.stopwords.words('english')
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
from bs4 import BeautifulSoup
import re

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_square_brackets(text):
    return re.sub('\[[^]]*\]','',text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_square_brackets(text)
    return text

imdb_data['review'] = imdb_data['review'].apply(denoise_text)
imdb_data.head()

<ipython-input-7-64e17961242f>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
def remove_special_charachters(text, remove_digits = True):
    pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern,'',text)
    return text

imdb_data['review'] = imdb_data['review'].apply(remove_special_charachters)
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [9]:
from nltk.stem.porter import PorterStemmer

def simple_stemmer(text):
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

imdb_data['review'] = imdb_data['review'].apply(simple_stemmer)
imdb_data.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,A wonder littl product the film techniqu is ve...,positive
2,I thought thi wa a wonder way to spend time on...,positive
3,basic there a famili where a littl boy jake th...,negative
4,petter mattei love in the time of money is a v...,positive


In [10]:
stop = set(stopwords.words('english'))
print(stop)

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

imdb_data['review'] = imdb_data['review'].apply(remove_stopwords)
imdb_data.head()

{'so', 'ma', 'above', 'weren', 'of', 'further', 'what', 'during', 'out', 'd', "hasn't", "you'll", "she's", "that'll", 'other', 'against', 'are', 'in', 'themselves', 'now', 'ours', 'yours', 'these', 'll', 'there', 'off', "aren't", "shouldn't", 'with', 'o', 'do', 'y', 'needn', 'as', 'the', "doesn't", 'it', 'being', "it's", "you'd", 'doing', "didn't", 'her', 'a', 'about', 'shan', 'then', 'both', 'will', 'is', 'did', 'be', 'to', "weren't", 'those', "wasn't", 'or', 'and', 'down', 'until', 'just', 'herself', "couldn't", 'we', 'been', 'under', 'has', 'from', 'up', 've', 'itself', 'aren', 'how', 'that', 'for', "isn't", 'hers', 'they', 'because', 'not', 'she', 'mustn', 'his', 'once', 'does', 'have', 'i', 'shouldn', 'where', 'if', 'you', "mightn't", 'ain', 's', 'between', 'nor', 'm', 'hasn', "mustn't", 'my', 'he', "you've", 'mightn', "wouldn't", 'each', 'were', 'any', 't', "you're", 'had', "should've", 'too', 'isn', 'same', 'here', 'don', 'having', 'all', 'himself', 'again', 'very', 'theirs', 'b

,review,sentiment
0,one review ha mention watch 1 Oz episod youll ...,positive
1,wonder littl product film techniqu veri unassu...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic famili littl boy jake think zombi hi clo...,negative
4,petter mattei love time money visual stun film...,positive


In [11]:
# pip install autocorrect

In [12]:
# from textblob import TextBlob
# from autocorrect import Speller

# norm_train_reviews=imdb_data.review[:1]
# print(norm_train_reviews)
# norm_train_string = norm_train_reviews.to_string()
# norm_train_spelling = Speller().spell(norm_train_string)
# # norm_train_spelling.correct()

# norm_train_words = norm_train_spelling.words
# norm_train_words
# print("-------------------------")
# print(norm_train_spelling)
# print("-------------------------")
# print(norm_train_words)